In [ ]:
import os
import re
import numpy as np
import pandas as pd
import os.path as op
import nibabel as nib
import matplotlib.pyplot as plt

from fury import actor, window
from dipy.io.streamline import load_tractogram

In [ ]:
plt.rcParams.update({
  "text.usetex": False,
  "font.family": "Helvetica",
  "font.size": 14
})

def lines_as_tubes(sl, linewidth, **kwargs):
  line_actor = actor.line(sl, **kwargs)
  line_actor.GetProperty().SetRenderLinesAsTubes(1)
  line_actor.GetProperty().SetLineWidth(linewidth)
  return line_actor

In [ ]:
paths_data = op.join("/path", "to", "data")
paths_save = op.join("paths", "to", "figure01")
os.makedirs(paths_save, exist_ok = True)

In [ ]:
# tract dice / overlap 
participant_info = [
  ["sub-XXX", "afq-XXX"], # low reliability
  ["sub-XXX", "afq-XXX"], # medium reliability
  ["sub-XXX", "afq-XXX"]  # high reliability
]
tract = "LeftInferiorLongitudinal"

for participant, method in participant_info: # for each participant
  # define the data directory
  data_dir = op.join(paths_data, participant)
  save_dir = op.join(paths_save, participant)
  os.makedirs(save_dir, exist_ok = True)

  # load glass brain mask
  glass_image = nib.load(op.join(data_dir, f"{participant}_space-ACPC_desc-glass_mask.nii.gz"))

  # load tractograms
  split_bname = f"{participant}_multi-shell_method-{method}_split-#_{tract}.trx"
  trks = {
    "split-1": load_tractogram(op.join(data_dir, re.sub("#", "1", split_bname)), glass_image),
    "split-2": load_tractogram(op.join(data_dir, re.sub("#", "2", split_bname)), glass_image),
  }

  # remove invalid streamlines
  for split in trks.keys(): 
    trks[split].remove_invalid_streamlines()

  # define glass brain actor
  glass_actor = actor.contour_from_roi(
    data    = glass_image.get_fdata(), 
    affine  = glass_image.affine, 
    color   = [0, 0, 0],
    opacity = 0.05
  )

  # open scene window
  scene = window.Scene()
  for split, curr_trk in trks.items(): # for each tractogram
    scene.clear() # clear scene
    scene.add(glass_actor) # add glass brain actor

    for sl in curr_trk.streamlines: # for each streamline
      sl_actor = lines_as_tubes([sl], linewidth = 7, colors = (0.65,) * 3)
      scene.add(sl_actor)

    # set camera position
    tract_direction = re.sub("^(Left|Right|Callosum)\\w+", "\\1", tract)
    match tract_direction: 
      case "Left":
        camera_kwargs = {
          "position": (-505.34, -99.47, 34.40), 
          "focal_point": (0.50, -16.50, 6.50), 
          "view_up": (0.07, -0.07, 1.00)
        }
      case "Right":
        camera_kwargs = {
          "position": (513.47, -19.48, -13.26), 
          "focal_point": (0.50, -16.50, 6.50),  
          "view_up": (0.04, 0.06, 1.00)
        }
      case "Callosum":
        camera_kwargs = {
          "position": (8.27, 27.84, 516.38), 
          "focal_point":  (-0.44, -3.52, 5.69), 
          "view_up": (-0.02, 1.00, -0.06)
        }

    scene.set_camera(**camera_kwargs)
    scene.background((1, 1, 1)) # white background

    # window.show(scene); scene.camera_info()
    save_name = f"{participant}_method-{method}_{split}_{tract}.png"
    window.record(
      scene = scene,
      out_path = op.join(save_dir, save_name),
      size = (2400, 2400)
    )
    print(f"Saved: {save_name}")

  scene.clear()

In [ ]:
# profile intraclass correlation
participant_info = [
  ["sub-XXX", "afq-XXX"], # low reliability
  ["sub-XXX", "afq-XXX"], # medium reliability
  ["sub-XXX", "afq-XXX"]  # high reliability
]
metric = "DKI-FA"
tract  = "LeftInferiorLongitudinal"

# y-axis aesthetics for DKI-FA (from figure09)
ylim = (0.05, 0.75); dy = 0.10; yticks = np.arange(ylim[0] + (dy/2), ylim[1], dy)

for participant, method in participant_info: # for each participant
  # read participant tract profiles
  data_dir = op.join("/path", "to", "reliprofiles")
  df_curr = pd.read_csv(op.join(data_dir, f"reliprofiles_{participant}.csv"))
  df_curr = df_curr[df_curr["dataset"] == "multi-shell"]
  df_curr = df_curr[df_curr["method"] == method]
  df_curr = df_curr[df_curr["metric"] == metric]
  df_curr = df_curr[df_curr["tract"] == tract]
  
  # format the dataframe for figure plotting
  df_plot = df_curr.pivot(
    index = "node", columns = "split", values = "value").reset_index()
  x = df_plot["node"].values; y1 = df_plot["split-1"].values; y2 = df_plot["split-2"].values
  
  fig, ax = plt.subplots(1, 1, figsize = (8, 4), tight_layout = True)
  ax.plot(x, y1, color = "black", linewidth = 3, label = "Split 1")
  ax.plot(x, y2, color = "blue", linewidth = 3, label = "Split 2")
  ax.set_xlim([-4.95, 103.95]); ax.set_ylim(ylim); ax.set_yticks(yticks)
  ax.set_xticks([]); ax.set_ylabel("DKI-FA")
  ax.set_title(f"{participant} ({method})")
  ax.legend()

  save_dir = op.join(paths_save, participant)
  os.makedirs(save_dir, exist_ok = True)
  
  save_name = f"{participant}_multi-shell_{method}_{metric}_{tract}.svg"
  fig.savefig(op.join(save_dir, save_name))
  print(f"Saved: {save_name}")
plt.show(); plt.close()